# Coursera Capstone Project
## By Federico Gómez
This notebook will be mainly used to develop the Capstone Project that makes part of the Data Science Specialty.

In [1]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

!conda install geopandas
import geopandas as gpd

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         154 KB

The following packages will be UPDATED:

    certifi:         2019.9.11-py36_0     conda-forge --> 2019.9.11-py36_0 
    openssl:         1.1.1c-h516909a_0    conda-forge --> 1.1.1d-h7b6447c_2

The following packages will be DOWNGRADED:

    ca-certificates: 2019.9.11-hecc5488_0 conda-forge --> 2019.8.28-0      


certifi-2019.9.11    | 154 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [11]:
geojson = gpd.read_file("https://data.sfgov.org/api/geospatial/pty2-tcw4?method=export&format=GeoJSON")

gdf = gpd.GeoDataFrame.from_features(geojson)
gdf['centroid_lon'] = gdf['geometry'].centroid.x
gdf['centroid_lat'] = gdf['geometry'].centroid.y
gdf.reset_index(inplace=True)
gdf['index'] = gdf['index']+1
gdf.head()

,index,geometry,link,name,centroid_lon,centroid_lat
0,1,(POLYGON ((-122.4934552679999 37.7835181710000...,"http://en.wikipedia.org/wiki/Sea_Cliff,_San_Fr...",Seacliff,-122.488844,37.786803
1,2,(POLYGON ((-122.4871507149999 37.7837854270000...,None,Lake Street,-122.479626,37.785513
2,3,(POLYGON ((-122.4775801709999 37.8109931130000...,http://www.nps.gov/prsf/index.htm,Presidio National Park,-122.466434,37.797416
3,4,(POLYGON ((-122.4724105299999 37.7873465390000...,None,Presidio Terrace,-122.464250,37.787007
4,5,(POLYGON ((-122.4726257899999 37.7863148060000...,http://www.sfgate.com/neighborhoods/sf/innerri...,Inner Richmond,-122.468504,37.779374


In [12]:
address = 'San Francisco, California'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of San Francisco are 37.7792808, -122.4192363.


In [13]:
# create map of San Francisco using latitude and longitude values
map_sf = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
map_sf.choropleth(
    geo_data=geojson,
    name='San Francisco neighborhoods',
    data=gdf,
    columns=['name', 'index'],
    fill_color='OrRd',
    key_on='feature.geometry.name',
    legend_name='index'
) 

# add markers with basic information
fg = folium.FeatureGroup(name='District Info')
for lat, lon, name, idx in zip(gdf['centroid_lat'].tolist(),
                               gdf['centroid_lon'].tolist(),
                               gdf['name'].tolist(),
                               gdf['index'].tolist()):
    html = f"""
    <p>{idx}. {name}<\p>
    """
    fg.add_child(folium.Marker(location=[lat, lon], popup=html))

map_sf.add_child(fg)

# enable layers to be turned 
folium.LayerControl().add_to(map_sf)

In [14]:
map_sf

In [15]:
# Hide this cell.
# Sensitive information
CLIENT_ID = 'XDPEXEBMOLXI3IOAARGTO2GF0JWLEXIQ1WESTHMKUXA2AXJW' # your Foursquare ID
CLIENT_SECRET = '4GYL1BQBEGTUHYUTIHZRD3SUN0FUNMPN1CHSJTMYX1HQRXSK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
sf_venues = getNearbyVenues(names = gdf['name'],
                            latitudes = gdf['centroid_lat'],
                            longitudes = gdf['centroid_lon']
                            )

Seacliff
Lake Street
Presidio National Park
Presidio Terrace
Inner Richmond
Sutro Heights
Lincoln Park / Ft. Miley
Outer Richmond
Golden Gate Park
Presidio Heights
Laurel Heights / Jordan Park
Lone Mountain
Anza Vista
Cow Hollow
Union Street
Nob Hill
Marina
Telegraph Hill
Downtown / Union Square
Tenderloin
Civic Center
Hayes Valley
Alamo Square
Panhandle
Haight Ashbury
Lower Haight
Mint Hill
Duboce Triangle
Cole Valley
Rincon Hill
South Beach
South of Market
Showplace Square
Mission Bay
Yerba Buena Island
Treasure Island
Mission Dolores
Castro
Outer Sunset
Parkside
Stonestown
Parkmerced
Lakeshore
Golden Gate Heights
Forest Hill
West Portal
Clarendon Heights
Midtown Terrace
Laguna Honda
Lower Nob Hill
Upper Market
Dolores Heights
Mission
Potrero Hill
Dogpatch
Central Waterfront
Diamond Heights
Crocker Amazon
Fairmount
Peralta Heights
Holly Park
Merced Manor
Balboa Terrace
Ingleside
Merced Heights
Outer Mission
Ingleside Terraces
Mt. Davidson Manor
Monterey Heights
Westwood Highlands
Wes

In [18]:
print(sf_venues.shape)
sf_venues.head()

(4883, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Seacliff,37.786803,-122.488844,China Beach,37.788090,-122.491186,Beach
1,Seacliff,37.786803,-122.488844,Seacliff,37.788259,-122.486401,Neighborhood
2,Seacliff,37.786803,-122.488844,Rochambeau Playground,37.784892,-122.484309,Playground
3,Seacliff,37.786803,-122.488844,Tal-Y-Tara Tea & Polo Shoppe,37.783710,-122.486750,Tea Room
4,Seacliff,37.786803,-122.488844,El Camino del Mar Trail,37.786568,-122.494399,Trail


In [19]:
sf_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Alamo Square,74,74,74,74,74,74
Anza Vista,17,17,17,17,17,17
Apparel City,27,27,27,27,27,27
Aquatic Park / Ft. Mason,64,64,64,64,64,64
Ashbury Heights,38,38,38,38,38,38
Balboa Terrace,12,12,12,12,12,12
Bayview,24,24,24,24,24,24
Bernal Heights,41,41,41,41,41,41
Bret Harte,6,6,6,6,6,6


In [20]:
print('There are {} uniques categories.'.format(len(sf_venues['Venue Category'].unique())))

There are 378 uniques categories.


In [21]:
# one hot encoding
sf_onehot = pd.get_dummies(sf_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sf_onehot['Neighbourhood'] = sf_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]

sf_onehot.head()

,Neighbourhood,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Credit Union,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nabe Restaurant,Nail Salon,National Park,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Physical Therapist,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Port,Portuguese Restaurant,Pub,Public Art,Ramen

In [22]:
sf_onehot.shape

(4883, 379)

In [23]:
sf_grouped = sf_onehot.groupby('Neighbourhood').mean().reset_index()
sf_grouped

,Neighbourhood,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Credit Union,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nabe Restaurant,Nail Salon,National Park,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Physical Therapist,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Port,Portuguese Restaurant,Pub,Public Art,Ramen

In [24]:
sf_grouped.shape

(117, 379)

In [25]:
a = list(sf_venues['Venue Category'].unique())
a.sort()
a

['ATM',
 'Acai House',
 'Accessories Store',
 'Adult Boutique',
 'Afghan Restaurant',
 'African Restaurant',
 'Alternative Healer',
 'American Restaurant',
 'Antique Shop',
 'Aquarium',
 'Arcade',
 'Arepa Restaurant',
 'Argentinian Restaurant',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Arts & Entertainment',
 'Asian Restaurant',
 'Athletics & Sports',
 'Austrian Restaurant',
 'Auto Garage',
 'Auto Workshop',
 'Automotive Shop',
 'BBQ Joint',
 'Baby Store',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Field',
 'Baseball Stadium',
 'Basketball Court',
 'Bath House',
 'Beach',
 'Bed & Breakfast',
 'Beer Bar',
 'Beer Garden',
 'Beer Store',
 'Big Box Store',
 'Bike Rental / Bike Share',
 'Bike Shop',
 'Bike Trail',
 'Bistro',
 'Board Shop',
 'Boat or Ferry',
 'Bookstore',
 'Boutique',
 'Bowling Alley',
 'Boxing Gym',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Bubble Tea Shop',
 'Building',
 'Burger Joint',
 'Burmese Restaurant',
 'Burrito Place',
 'Bu

In [26]:
num_top_venues = 5

for hood in sf_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = sf_grouped[sf_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alamo Square----
                  venue  freq
0                   Bar  0.05
1          Liquor Store  0.04
2      Sushi Restaurant  0.03
3           Record Shop  0.03
4  Ethiopian Restaurant  0.03


----Anza Vista----
                     venue  freq
0                     Café  0.18
1  Health & Beauty Service  0.12
2             Burger Joint  0.06
3           Sandwich Place  0.06
4       Mexican Restaurant  0.06


----Apparel City----
                 venue  freq
0            Nightclub  0.07
1           Food Truck  0.07
2  Arts & Crafts Store  0.07
3   Seafood Restaurant  0.07
4              Brewery  0.04


----Aquatic Park / Ft. Mason----
                 venue  freq
0           Food Truck  0.06
1          Art Gallery  0.06
2  Arts & Crafts Store  0.05
3       Scenic Lookout  0.05
4              Theater  0.05


----Ashbury Heights----
            venue  freq
0            Park  0.11
1  Breakfast Spot  0.05
2        Wine Bar  0.05
3           Trail  0.05
4     Coffee Shop  0.05


--

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = sf_grouped['Neighbourhood']

for ind in np.arange(sf_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alamo Square,Bar,Liquor Store,Seafood Restaurant,Café,Ethiopian Restaurant,Record Shop,Pizza Place,Wine Bar,Sushi Restaurant,Hotel
1,Anza Vista,Café,Health & Beauty Service,Cosmetics Shop,Arts & Crafts Store,Big Box Store,Tunnel,Mexican Restaurant,Sandwich Place,Bus Stop,Grocery Store
2,Apparel City,Food Truck,Nightclub,Seafood Restaurant,Arts & Crafts Store,Garden Center,Gas Station,Miscellaneous Shop,Mexican Restaurant,Automotive Shop,Outdoor Supply Store
3,Aquatic Park / Ft. Mason,Art Gallery,Food Truck,Theater,Arts & Crafts Store,Scenic Lookout,Vegetarian / Vegan Restaurant,Chocolate Shop,Park,Café,National Park
4,Ashbury Heights,Park,Trail,Breakfast Spot,Wine Bar,Coffee Shop,Bakery,Toy / Game Store,Grocery Store,Bar,Gift Shop
5,Balboa Terrace,Light Rail Station,Park,Dessert Shop,Vietnamese Restaurant,Comic Shop,Playground,American Restaurant,Fountain,Gym,Pharmacy
6,Bayview,Southern / Soul Food Restaurant,Bakery,Convenience Store,Mexican Restaurant,Fried Chicken Joint,African Restaurant,Market,Garden,Light Rail Station,Park
7,Bernal Heights,Park,Playground,Coffee Shop,Italian Restaurant,Trail,Bakery,Gourmet Shop,Seafood Restaurant,Bus Stop,Cocktail Bar
8,Bret Harte,Light Rail Station,Park,Brewery,Burger Joint,Distillery,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant
9,Buena Vista,Park,Tennis Court,Breakfast Spot,Scenic Lookout,Museum,Dog Run,Trail,Sandwich Place,Clothing Store,Historic Site


## Explanation:
Here we define a list of places related with partying and search for the neighborhoods where these kind of venues thrive the most.

In [38]:
search_df = pd.DataFrame(neighbourhoods_venues_sorted)

In [46]:
places = ['Bar', 'Gay Bar', 'Wine Bar', 'Liquor Store', 'Cocktail Bar']
search_df.loc[(search_df['1st Most Common Venue'].isin(places)) | (search_df['2nd Most Common Venue'].isin(places)) | (search_df['3rd Most Common Venue'].isin(places))]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alamo Square,Bar,Liquor Store,Seafood Restaurant,Café,Ethiopian Restaurant,Record Shop,Pizza Place,Wine Bar,Sushi Restaurant,Hotel
11,Castro,Gay Bar,Coffee Shop,Thai Restaurant,Men's Store,Pet Store,New American Restaurant,Cosmetics Shop,Juice Bar,Gym,Café
16,Civic Center,Cocktail Bar,Coffee Shop,Café,Theater,French Restaurant,Boutique,Concert Hall,Vegetarian / Vegan Restaurant,Taco Place,Beer Bar
19,Corona Heights,Gay Bar,Park,Cosmetics Shop,Scenic Lookout,Thai Restaurant,Pizza Place,Plaza,Tennis Court,Deli / Bodega,Furniture / Home Store
24,Dolores Heights,Shoe Store,Liquor Store,Park,Coffee Shop,Turkish Restaurant,Gastropub,Café,Grocery Store,Sushi Restaurant,Art Gallery
27,Eureka Valley,Gay Bar,Playground,Pet Store,Scenic Lookout,Thai Restaurant,Coffee Shop,Park,Convenience Store,Yoga Studio,Grocery Store
38,Hayes Valley,Boutique,Wine Bar,Clothing Store,French Restaurant,Dessert Shop,Café,Pizza Place,Sushi Restaurant,Cocktail Bar,Optical Shop
55,Lower Nob Hill,Coffee Shop,Cocktail Bar,Bar,Breakfast Spot,Sushi Restaurant,Café,Thai Restaurant,Gym / Fitness Center,Speakeasy,Art Gallery
62,Mint Hill,Sushi Restaurant,Gym / Fitness Center,Wine Bar,Café,Grocery Store,Mexican Restaurant,Cocktail Bar,Ramen Restaurant,Pet Store,Boutique
108,Union Street,Cosmetics Shop,Bar,Spa,Mexican Restaurant,Yoga Studio,Café,Wine Bar,Italian Restaurant,Park,Sandwich Place


In [47]:
top_hoods = pd.DataFrame(search_df.loc[(search_df['1st Most Common Venue'].isin(places)) | (search_df['2nd Most Common Venue'].isin(places)) | (search_df['3rd Most Common Venue'].isin(places))])['Neighbourhood']
top_hoods

0          Alamo Square
11               Castro
16         Civic Center
19       Corona Heights
24      Dolores Heights
27        Eureka Valley
38         Hayes Valley
55       Lower Nob Hill
62            Mint Hill
108        Union Street
109    University Mound
113    Western Addition
Name: Neighbourhood, dtype: object